In [1]:
# Load the csv file to pandas
import pandas as pd
df=pd.read_csv("balanced_risk_unique_body_priority_800.csv")
df.head()

body priority
0       High CPU usage detected on critical server.   Medium
1           Service level agreement breach warning.      Low
2       Critical security vulnerability identified.      Low
3  Disk I/O performance issues under investigation.   Medium
4            Low disk space alert, cleanup advised.     High

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   body      800 non-null    object
 1   priority  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


In [15]:
# Display column names
cols=df.columns
cols

Index(['body', 'priority'], dtype='object')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   body      800 non-null    object
 1   priority  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


In [6]:
import pandas as pd

# Finding duplicates
duplicates = df[df.duplicated()]

# This will show all rows that are duplicates (excluding the first occurrence)
print(duplicates)


                                                  body priority
90         High CPU usage detected on critical server.   Medium
91             Service level agreement breach warning.      Low
92         Critical security vulnerability identified.      Low
93    Disk I/O performance issues under investigation.   Medium
94              Low disk space alert, cleanup advised.     High
..                                                 ...      ...
795        Security patch applied, monitoring ongoing.      Low
796  Performance degradation observed, investigatio...     High
797      User access revoked due to security concerns.   Medium
798        Firewall rules updated, monitoring traffic.   Medium
799  Configuration mismatch detected, correcting au...     High

[710 rows x 2 columns]


In [7]:
df=df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   body      90 non-null     object
 1   priority  90 non-null     object
dtypes: object(2)
memory usage: 2.1+ KB


In [8]:
# Check the frequency of each category for the columns
df['priority'].value_counts()


priority
Medium    30
Low       30
High      30
Name: count, dtype: int64

In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['priority'] = label_encoder.fit_transform(df['priority'])



In [10]:
# Split data into training & test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['body'], df['priority'], test_size=0.2, random_state=42, stratify=df["priority"])



In [11]:
X_train

23          High CPU usage detected on critical server.
38             Automated task execution failure logged.
52      Network congestion detected, rerouting traffic.
62        User access revoked due to security concerns.
61         Unexpected system reboot, logs under review.
                            ...                        
35           Power supply failure, switching to backup.
68                Unauthorized access attempt detected.
20    Routine maintenance check completed successfully.
81      Network congestion detected, rerouting traffic.
28     Backup verification failed, initiating recovery.
Name: body, Length: 72, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [13]:
# Train Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train_tfidf, y_train)

# Extracting the best model from grid search cv
best_rf_model = grid_search.best_estimator_




Fitting 5 folds for each of 243 candidates, totalling 1215 fits


In [17]:
y_predict = best_rf_model.predict(X_test_tfidf)

In [19]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_predict, target_names=["High", "Low", "Medium"])
print(report)


              precision    recall  f1-score   support

        High       0.21      0.50      0.30         6
         Low       0.25      0.17      0.20         6
      Medium       0.00      0.00      0.00         6

    accuracy                           0.22        18
   macro avg       0.15      0.22      0.17        18
weighted avg       0.15      0.22      0.17        18



c:\Users\Karthik\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Karthik\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Karthik\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [22]:
import joblib
joblib.dump(best_rf_model, 'model_file.pkl')


['model_file.pkl']